# Baseline Annotation of Observation based on trained spaCy model 


<fig size= "4">
This notebook provides a baseline annotation suggestion by using refined named entity recognition that we provided using training a deep learning method by prodigy.  </p>
</fig size= "4">  

In [ ]:
import spacy 
from spacy import displacy 
from __future__ import print_function
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)


## Read the Data

In [ ]:
import pandas as pd
import numpy as np
importVersion = '013'

In [ ]:
path= '../data/01_df_v{0}.pickle'.format(importVersion)# Put the path of the data in your local machine here, consider the letter "r" before the path
dfAstroNova = pd.read_pickle(path)

In [ ]:
# Sort the data based on the chapters of the book 
dfAstroNova['chapter'] = dfAstroNova.chapter.replace("appendix b",np.nan).astype(float)  
dfAstroNova = dfAstroNova.rename_axis('MyIdx').sort_values(by = ['chapter', 'MyIdx'], ascending = [True, True])
dfAstroNova.chapter.fillna('appendix b', inplace=True)

In [ ]:
dfAstroNova.reset_index(inplace=True)
dfAstroNova=dfAstroNova.drop("MyIdx",axis=1,inplace=False)
dfAstroNova=dfAstroNova.drop("html",axis=1)

In [ ]:
type(dfAstroNova)

In [ ]:
dfAstroNova.head()

In [ ]:
df=dfAstroNova.reset_index().text.str.split('.',expand=True).stack().reset_index(level=-1,drop=True)
type(df)

In [ ]:
 # Create lists to fill with values
l_col1 = []
l_col2 = []

# iterrate over each row and fill our lists
for ix, row in dfAstroNova.iterrows():
    for value in row['sentences']:
        l_col1.append(value)
        l_col2.append(row['chapter'])

# Create new dataframe from the two lists
df= pd.DataFrame({'sentences': l_col1 ,
                         'chapter': l_col2 })
df=df.rename(columns={"sentences":"sents"});

In [ ]:
len(df)

In [ ]:
df.head()

# Upload customized spaCy model provioded by our training data in prodigy

In [ ]:
path= '../data/Model_11'
nlp=spacy.load(path)

In [ ]:
def entity(x):
    a=[]
    b=[]
    for ent in nlp(x).ents:
        a +=ent.text,
        b +=ent.label_,
    c=list(zip(a,b))
    return c

In [ ]:
def noun_chunk(x):
    a=[]
    for chunk in nlp(x).noun_chunks:
        a.append(chunk)
    return a

In [ ]:
df['entities'] = df['sents'].apply(lambda x: entities(x))

In [ ]:
df['chunks'] = df['sents'].apply(lambda x: noun_chunk(x))

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
def annot_observation_01(x):
    y=0
    if x is not None:
        for idx in range(0,len(x)):
                    if any ([x[idx][1] == 'DATE', x[idx][1] == 'LONG', x[idx][1] == 'TIME']):
                        y=1
                    else:
                        y=0   
    else:
        y=0
    return y
                    
       

    

In [ ]:
df['label'] = df['entities'].apply(lambda x: annot_observation_01(x))

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df[df.chapter==10]

In [ ]:
df.sents[1]

In [ ]:
def show_chapter(x):
     return df.loc[df.chapter==x]



In [ ]:
len(df)

In [ ]:
interact(show_chapter,x=(1,70,1))
         

In [ ]:
def depen_interact(x):
    c=displacy.render(nlp(df.sents[x]),style="dep",jupyter="True")
    return c


In [ ]:
interact(depen_interact, x=(0,len(df),1));

# Idea of using SDP

In [ ]:
import spacy
import networkx as nx

doc = nlp(u'where the equation caused by the eccentricity is maximum. ')
print('sentence:'.format(doc))
# Load spacy's dependency tree into a networkx graph
edges = []
for token in doc:
    for child in token.children:
        edges.append(('{0}'.format(token.lower_),
                      '{0}'.format(child.lower_)))
graph = nx.Graph(edges)
# Get the length and path
entity1 = 'equation'.lower()
entity2 = 'eccentricity'
print(nx.shortest_path_length(graph, source=entity1, target=entity2))
print(nx.shortest_path(graph, source=entity1, target=entity2))

In [ ]:
displacy.render(doc,style="dep",jupyter="True" )

In [ ]:
edges

In [ ]:
graph.is_directed()

In [ ]:
graph.number_of_nodes()

In [ ]:
graph.number_of_edges()

In [ ]:
graph.degree

In [ ]:
for token in doc:
    print((token.head.text, token.text, token.dep_))